In [ ]:
# Initialization cell
import os, json, platform, random, logging, sys, math, csv, zipfile, shutil
from pathlib import Path
from timeit import default_timer as timer

try:
    import pandas as pd
except:
    !pip install -q pandas
    import pandas as pd

try:
    from tqdm.auto import tqdm
except:
    !pip install -q tqdm
    from tqdm.auto import tqdm

try:
    import IPython
except:
    !pip install -q IPython
    import IPython

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer, AdamW
from torch.optim.lr_scheduler import LambdaLR

# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Hints:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer.


# Added by HL

## Check if GPU available (added by HL)

In [ ]:
import platform
try:
    import torch
    if platform.system() != 'Darwin':
        if torch.cuda.is_available():
            !nvidia-smi
        else:
            print("GPU not found")
except:
    print("""Torch is not available.
Please install by:
`conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia`""")


Wed Aug 16 07:22:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
try:
    import torch
    device = "mps" if platform.system() == 'Darwin' and torch.backends.mps.is_built() \
            else "cuda" if torch.cuda.is_available() else "cpu"
except:
    device = "cpu"

device == "cuda" or device == "mps" or print("GPU is currently NOT available for PyTorch.\nPlease reinstall PyTorch if you want to run in GPU.") # check if GPU is available for PyTorch
device

'cuda'

## Logger (added by HL)

In [ ]:
import logging
import sys

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                    level=logging.INFO, datefmt='%Y-%m-%d %H:%M:%S',
                    stream=sys.stdout)
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='myelog.log', mode='a')
formatter = logging.Formatter('%(asctime)s | %(levelname)s : %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

## Download / Unzip data (added by HL)

In [ ]:
import zipfile
from pathlib import Path
import shutil


def download_data():
    # Setup path to data folder
    data_path = Path("Dataset")
    zip_path = Path("Dataset.zip")

    def unzip_file(zip_path):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            logging.info('Unzipping dataset...')
            zip_ref.extractall()
        logging.info("Data unzipped.")
        if Path('__MACOSX').exists():
            shutil.rmtree('__MACOSX')

    # If the image folder doesn't exist, download it and prepare it...
    if data_path.is_dir():
        print(f"{data_path} directory exists.")
    elif zip_path.exists():
        print(f"{zip_path} exists, now unzip.")
        unzip_file(zip_path)
    else:
        print(f"{zip_path} does not exist, now download from drive.")
        logging.info('Downloading dataset...')
        try:
            import gdown
        except:
            !pip install -q gdown
            import gdown
        from Dataset import dataset_link
        url = dataset_link()
        output = 'Dataset.zip'
        gdown.download(url, output, quiet=False)
        logging.info("Data downloaded, now unzip.")
        unzip_file(zip_path)

download_data()

Dataset directory exists.


## Timer (added by HL)

In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    
    hour = int(total_time//3600)
    min = int((total_time%3600)//60)
    sec = (total_time%60)

    total_time_str = f"{sec:.2f} second{'s' if sec > 1 else ''}" if total_time < 60 else f"{min} minute{'s' if min > 1 else ''} and {sec} second{'s' if sec > 1 else ''}" if total_time < 3600 else f"{hour} hour{'s' if hour > 1 else ''} {min} minute{'s' if min > 1 else ''} {sec} second{'s' if sec > 1 else ''}"

    print(f'Train time on {device}: {total_time_str}')
    return total_time_str

## results_df (added by HL)

In [ ]:
try:
    import pandas as pd
except:
    !pip install -q pandas
    import pandas as pd
import torch
from pathlib import Path

result_path = Path("results.xlsx")
if result_path.exists():
    results_df = pd.read_excel("results.xlsx")
else:
    results_df = pd.DataFrame(columns=['device', 'model', 'transformer/conformer', 'attention', 'batch_size', 'num_workers', 'accuracy', 'train_time'])

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- 600 randomly selected speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory
  - data directory <br>
  |---- metadata.json <br>
  |---- testdata.json <br>
  |---- mapping.json <br>
  |---- uttr-{random string}.pt

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker name to their corresponding id.
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start+self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install -q conformer

class Classifier(nn.Module):
    def __init__(self, d_model=80, n_spks=600, dropout=0.1, use_conformer=False):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
        if use_conformer:
            from conformer import ConformerBlock
            
            self.encoder_layer = ConformerBlock(
                dim=d_model, dim_head=64, heads=2, ff_mult=4,
                conv_expansion_factor=2, conv_kernel_size=3,
                attn_dropout=dropout, ff_dropout=dropout,
                conv_dropout=dropout
			)
        else:
            self.encoder_layer = nn.TransformerEncoderLayer(
				d_model=d_model, dim_feedforward=256, nhead=2
			)
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, n_spks),
        )

    def forward(self, mels):
        """
        args:
          mels: (batch size, length, 40)
        return:
          out: (batch size, n_spks)
        """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        # mean pooling
        stats = out.mean(dim=1)

        # out: (batch, n_spks)
        out = self.pred_layer(stats)
        return out


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [ ]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

    Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """

    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(
            0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
        )

    return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [ ]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [ ]:
from tqdm.auto import tqdm
import torch


def valid(dataloader, model, criterion, device):
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)


### You may try this before main function (added, to be removed)
You can run the following code to check if `iter(train_loader)` works.  

---
```python
data_dir = "./Dataset"
batch_size = 32
n_workers = 8

train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
train_iterator = iter(train_loader)
```
---

The `n_workers` is `8` in the original code, but I found it does not work in local machines unless it is changed to 0 .<br>
However, if it can run properly in your local machine, you may try running the main function by:
```python
    main(**parse_args(n_workers=8))
```

---

# Main function

In [ ]:
from tqdm.auto import tqdm
import platform

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args(batch_size=32, n_workers=8 if 'google.colab' in str(get_ipython()) else 0, use_conformer=False, use_attention=False):
    """arguments"""
    config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": batch_size,
    "n_workers": n_workers,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 60000,
    "use_conformer": use_conformer, # added by HL
    "use_attention": use_attention, # added by HL
    }

    return config

def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
    use_conformer,
    use_attention,
    classifier=Classifier,
):
    """Main function."""
    # set device to mps if mac, cuda if windows/linux, else cpu
    device = "mps" if platform.system() == 'Darwin' and torch.backends.mps.is_built() \
                else "cuda" if torch.cuda.is_available() else "cpu"
    logging.info(f'Use {device} now!')

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    logging.info(f"[Info]: Finish loading data!", extra={'flush': True})

    model = classifier(n_spks=speaker_num, use_conformer=use_conformer).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    logging.info(f"[Info]: Finish creating model!", extra={'flush': True})

    best_accuracy = -1.0
    best_state_dict = None

    train_time_start = timer()

    pbar = tqdm(total=valid_steps, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()

    train_time_end = timer()
    total_train_time_model = print_train_time(start=train_time_start,
                                           end=train_time_end,
                                           device=str(next(model.parameters()).device))

    return model, use_conformer, use_attention, batch_size, n_workers, best_accuracy, total_train_time_model


#### Batch_size = 32

In [ ]:
if __name__ == "__main__":
    model, use_conformer, use_attention, batch_size, n_workers, accuracy, train_time = main(**parse_args(batch_size=32, use_conformer=False, use_attention=False), classifier=Classifier)
    results_df.loc[len(results_df)] = ["MPS" if device == 'mps' else torch.cuda.get_device_name() if device == 'cuda' else 'cpu', str(model), 'conformer' if use_conformer else 'transformer', 'Y' if use_attention else 'N', batch_size, n_workers, accuracy, train_time]

#### Batch_size = 64

In [ ]:
if __name__ == "__main__":
    model, use_conformer, use_attention, batch_size, n_workers, accuracy, train_time = main(**parse_args(batch_size=64, use_conformer=False, use_attention=False), classifier=Classifier)
    results_df.loc[len(results_df)] = ["MPS" if device == 'mps' else torch.cuda.get_device_name() if device == 'cuda' else 'cpu', str(model), 'conformer' if use_conformer else 'transformer', 'Y' if use_attention else 'N', batch_size, n_workers, accuracy, train_time]

#### Batch_size = 128

In [ ]:
if __name__ == "__main__":
    model, use_conformer, use_attention, batch_size, n_workers, accuracy, train_time = main(**parse_args(batch_size=128, use_conformer=False, use_attention=False), classifier=Classifier)
    results_df.loc[len(results_df)] = ["MPS" if device == 'mps' else torch.cuda.get_device_name() if device == 'cuda' else 'cpu', str(model), 'conformer' if use_conformer else 'transformer', 'Y' if use_attention else 'N', batch_size, n_workers, accuracy, train_time]

# Results (added by HL)

In [ ]:
display(results_df)
try:
    results_df.to_excel('results.xlsx', index = False)
except:
    !pip install -q openpyxl
    results_df.to_excel('results.xlsx', index = False)
print("\nResults is saved into an xlsx file")

# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)


## Main function of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm
import platform

import torch
from torch.utils.data import DataLoader

def parse_args(use_conformer=False):
    """arguments"""
    config = {
    "data_dir": "./Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
    "use_conformer": use_conformer, # added by HL
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
    use_conformer,
    ):
    """Main function."""
    device = "mps" if platform.system() == 'Darwin' and torch.backends.mps.is_built() \
            else "cuda" if torch.cuda.is_available() else "cpu"
    logging.info(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8 if 'google.colab' in str(get_ipython()) else 0,
    collate_fn=inference_collate_batch,
    )
    logging.info(f"[Info]: Finish loading model!", extra={'flush': True})

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(n_spks=speaker_num, use_conformer=use_conformer).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    logging.info(f"[Info]: Finish creating model!", extra={'flush': True})

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            outs = model(mels)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


In [ ]:
if __name__ == "__main__":
    main(**parse_args(use_conformer=False))

# Zip all files (added by HL)

In [ ]:
import os
import zipfile

path = os.getcwd()  # get current working directory
file_names = []

for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
        file_names.append(file)

with zipfile.ZipFile('files_send.zip','w') as zip:
    # writing each file one by one
    for file in file_names:
        zip.write(file)

print('All files zipped successfully!')